In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import boto3
from py_dreambooth.dataset import AWSDataset
from py_dreambooth.model import SdxlDreamboothLoraModel
from py_dreambooth.trainer import AWSTrainer
from py_dreambooth.utils.image_helpers import display_images
from py_dreambooth.utils.prompt_helpers import make_prompt

In [ ]:
DATA_DIR = "data"

boto_session = boto3.Session()
dataset = AWSDataset(DATA_DIR, boto_session)

In [ ]:
dataset = dataset.preprocess_images(detect_face=True)
dataset = dataset.upload_images()

In [ ]:
SUBJECT_NAME = "<YOUR-NAME>"
CLASS_NAME = "person"

model = SdxlDreamboothLoraModel(
    subject_name=SUBJECT_NAME, 
    class_name=CLASS_NAME, 
    seed=42, 
    max_train_steps=1200,
)

trainer = AWSTrainer(
    report_to="wandb",
    wandb_api_key="<YOUR-WANDB-API-KEY>",
    sm_train_instance_type="ml.g5.2xlarge",
    sm_infer_instance_type="ml.g4dn.2xlarge",
)

In [ ]:
%%time
predictor = trainer.fit(model, dataset)

In [ ]:
%%time
prompt = f"A photo of {SUBJECT_NAME} {CLASS_NAME} with Eiffel Tower in the background"
# prompt = next(make_prompt(SUBJECT_NAME, CLASS_NAME))

print(f"The prompt is as follows:\n{prompt}")

images = predictor.predict(
    prompt, height=768, width=512, num_images_per_prompt=2, seed=42
)

display_images(images, fig_size=10)

In [ ]:
predictor.delete_endpoint()